In [1]:
import pandas as pd
import numpy as np

In [33]:
path = '/Users/jpmvbastos/Library/CloudStorage/OneDrive-TexasTechUniversity/Fall 2023/Causal Inference/Term Paper/Data/IPEA/'
df_list = ['bankbranches','bankdeposits','homiciderate','icms_transfers','pibmunicipal','savings','taxrevenue','population'] 
for i in df_list:
    x = pd.read_excel(path+i+".xls")
    x = pd.melt(x, id_vars=['Sigla','Codigo','Município'], var_name='Year', value_name=i) # Use pd.melt() to convert to long format
    x.sort_values(['Município', 'Year'], inplace=True)
    globals()[i] = x
    
    


In [100]:
df = bankbranches
for i in [bankdeposits,homiciderate,icms_transfers,pibmunicipal,savings,taxrevenue,population]:
    df = pd.merge(df, i, on=['Sigla','Codigo','Município','Year'], suffixes=('','right'))
df.Year = df.Year.astype(int)
df['Município'] = df['Município'].apply(lambda x: x.upper())
df

,Sigla,Codigo,Município,Year,bankbranches,bankdeposits,homiciderate,icms_transfers,pibmunicipal,savings,taxrevenue,population
0,GO,5200050,ABADIA DE GOIÁS,2003,NaN,NaN,17.580872,786765.74,37920.338097,NaN,251528.623150,5621.0
1,GO,5200050,ABADIA DE GOIÁS,2004,NaN,NaN,16.963528,934511.02,36710.710950,NaN,339947.964205,6054.0
2,GO,5200050,ABADIA DE GOIÁS,2005,NaN,NaN,47.664442,928955.40,40500.359822,NaN,349370.706103,6294.0
3,GO,5200050,ABADIA DE GOIÁS,2006,NaN,NaN,15.883100,815089.56,41983.969137,NaN,534258.015601,6531.0
4,GO,5200050,ABADIA DE GOIÁS,2008,NaN,NaN,64.703979,1238956.84,47333.365220,NaN,587395.303467,6182.0
...,...,...,...,...,...,...,...,...,...,...,...,...
78339,SP,3533809,ÓLEO,2014,NaN,NaN,NaN,2859740.22,56968.400922,NaN,507893.396807,2652.0
78340,SP,3533809,ÓLEO,2015,NaN,NaN,NaN,3032534.69,57057.341856,NaN,398603.707587,2628.0
78341,SP,3533809,ÓLEO,2016,NaN,NaN,NaN,3215613.05,57017.611839,NaN,479869.911168,2605.0
78342,SP,3533809,ÓLEO,2017,NaN,NaN,NaN,3498748.00,54150.480409,NaN,540323.620674,2583.0


## Airport Data

In [52]:
air = pd.read_excel('/Users/jpmvbastos/Library/CloudStorage/OneDrive-TexasTechUniversity/Fall 2023/Causal Inference/Term Paper/Data/AirportData.xlsx')

In [96]:
air.columns

Index(['EMPRESA (SIGLA)', 'EMPRESA (NOME)', 'EMPRESA (NACIONALIDADE)', 'ANO',
       'MÊS', 'AEROPORTO DE ORIGEM (SIGLA)', 'AEROPORTO DE ORIGEM (NOME)',
       'AEROPORTO DE ORIGEM (UF)', 'AEROPORTO DE ORIGEM (REGIÃO)',
       'AEROPORTO DE ORIGEM (PAÍS)', 'AEROPORTO DE ORIGEM (CONTINENTE)',
       'AEROPORTO DE DESTINO (SIGLA)', 'AEROPORTO DE DESTINO (NOME)',
       'AEROPORTO DE DESTINO (UF)', 'AEROPORTO DE DESTINO (REGIÃO)',
       'AEROPORTO DE DESTINO (PAÍS)', 'AEROPORTO DE DESTINO (CONTINENTE)',
       'NATUREZA', 'GRUPO DE VOO', 'PASSAGEIROS PAGOS', 'PASSAGEIROS GRÁTIS',
       'CARGA PAGA (KG)', 'CARGA GRÁTIS (KG)', 'CORREIO (KG)', 'ASK', 'RPK',
       'ATK', 'RTK', 'COMBUSTÍVEL (LITROS)', 'DISTÂNCIA VOADA (KM)',
       'DECOLAGENS', 'CARGA PAGA KM', 'CARGA GRATIS KM', 'CORREIO KM',
       'ASSENTOS', 'PAYLOAD', 'HORAS VOADAS', 'BAGAGEM (KG)'],
      dtype='object')

In [97]:
airp = air.loc[(air['AEROPORTO DE DESTINO (PAÍS)']=='BRASIL')
                    ].groupby(['AEROPORTO DE DESTINO (NOME)','AEROPORTO DE DESTINO (UF)','ANO']).agg(
                        {'AEROPORTO DE ORIGEM (PAÍS)':'nunique', # Number of Countries it receives flights from
                              'AEROPORTO DE ORIGEM (NOME)':'count', # Number of Airports that it receives flights from 
                              'PASSAGEIROS PAGOS':'sum',}) # Number of Passengers
airp.rename(columns={'AEROPORTO DE ORIGEM (PAÍS)':'ncountry_from',
                     'AEROPORTO DE ORIGEM (NOME)':'nairports_from',
                     'PASSAGEIROS PAGOS':'npassengers'}, inplace=True)
airp.reset_index(inplace=True)

In [101]:
df = pd.merge(df, airp, left_on=['Município','Sigla','Year'], right_on=['AEROPORTO DE DESTINO (NOME)','AEROPORTO DE DESTINO (UF)','ANO'], how='left')
df.loc[df['npassengers'].isnull(), ['ncountry_from','nairports_from','npassengers']] = 0 # Input zero values for cities without airports

In [112]:
df.loc[df['Município']=='SÃO PAULO']

,Sigla,Codigo,Município,Year,bankbranches,bankdeposits,homiciderate,icms_transfers,pibmunicipal,savings,taxrevenue,population,AEROPORTO DE DESTINO (NOME),AEROPORTO DE DESTINO (UF),ANO,ncountry_from,nairports_from,npassengers
70168,SP,3550308,SÃO PAULO,2003,1974.0,2.665523e+07,46.715354,2.547455e+09,3.436218e+08,3.812278e+07,1.174771e+10,10677019.0,NaN,NaN,NaN,0.0,0.0,0.0
70169,SP,3550308,SÃO PAULO,2004,1982.0,2.877482e+07,36.609451,2.828785e+09,3.397577e+08,3.760917e+07,1.275290e+10,10838581.0,NaN,NaN,NaN,0.0,0.0,0.0
70170,SP,3550308,SÃO PAULO,2005,2031.0,3.132795e+07,26.228364,3.049896e+09,3.666180e+08,3.644491e+07,1.392228e+10,10927985.0,NaN,NaN,NaN,0.0,0.0,0.0
70171,SP,3550308,SÃO PAULO,2006,2139.0,3.911027e+07,23.878682,3.398328e+09,3.823800e+08,3.682580e+07,1.671659e+10,11016703.0,NaN,NaN,NaN,0.0,0.0,0.0
70172,SP,3550308,SÃO PAULO,2008,2323.0,4.048819e+07,15.646408,4.483833e+09,4.119928e+08,4.388935e+07,1.851987e+10,10990252.0,SÃO PAULO,SP,2008.0,4.0,1665.0,6394142.0
70173,SP,3550308,SÃO PAULO,2009,2514.0,4.301878e+07,16.061728,4.598209e+09,4.226518e+08,5.020955e+07,1.882829e+10,11037593.0,NaN,NaN,NaN,0.0,0.0,0.0
70174,SP,3550308,SÃO PAULO,2011,2465.0,3.672711e+07,13.096302,5.815902e+09,4.634148e+08,5.400019e+07,2.391799e+10,11316119.0,NaN,NaN,NaN,0.0,0.0,0.0
70175,SP,3550308,SÃO PAULO,2012,2482.0,3.993247e+07,16.201646,6.152986e+09,4.608841e+08,5.910950e+07,2.629689e+10,11376685.0,NaN,NaN,NaN,0.0,0.0,0.0
70176,SP,3550308,SÃO PAULO,2013,2554.0,3.762634e+07,14.659268,6.944285e+09,4.630816e+08,6.674197e+07,2.589830e+10,11821873.0,NaN,NaN,NaN,0.0,0.0,0.0
70177,SP,3550308,SÃO PAULO,2014,2586.0,3.346971e+07,15.408678,6.802640e+09,4.587762e+08,6.781772e+07,2.694982e+10,11895893.0,NaN,NaN,NaN,0.0,0.0,0.0


In [51]:
# FIFA announced the host cities in 31 Maio 2009
host = ['Rio de Janeiro', 'São Paulo', 'Belo Horizonte', 'Porto Alegre', 'Curitiba', 
            'Brasília', 'Fortaleza', 'Salvador', 'Recife', 'Natal', 'Cuiabá', 'Manaus']
candidates = ['Rio de Janeiro', 'São Paulo', 'Belo Horizonte', 'Porto Alegre', 'Curitiba', 
              'Brasília', 'Fortaleza', 'Salvador', 'Recife', 'Natal', 'Cuiabá', 'Manaus',
              'Florianópolis', 'Goiânia', 'Campo Grande', 'Rio Branco', 'Belém']

df['host'] = np.where(df['Município'].isin(host), 1, 0) 
df['cand'] = np.where(df['Município'].isin([candidates]), 1, 0) 


